In [ ]:
#En este apartado voy a dejar varias piezas de código que no tienen aplicación directa en la versión final de este proyecto
#La gran mayoría de ellos tienen que ver con la idea original alrededor de la cual formamos el equipo
#Incluyo también una descripción de esas fases del proyecto:


#Idea general: Crear un algoritmo con el que hacer que, en tiempo real, un usuario pueda ver tanto la letra como los acordes en 
#cifrado americano de una canción.

#PRIMER PLANTEAMIENTO. Investigación

# - Lo primero que planteamos fue buscar una API que nos permitiera sacar la letra de una canción.
# - Por otro lado, necesitamos obtener los acordes de esa misma canción de algún otro sitio.
#     - Esto último no era tan sencillo, así qeu voy a mencionar algunas de las ideas que barajamos:
#            - Buscar una API desde la que obtener los acordes
#            - Buscar una base de datos que contuviera a la vez la letra y los acordes, aunque por separado
#            - Entrenar un modelo para que fuera capaz de escuchar la canción y sacar los acordes directamente

#Esto último era muy complejo desde el punto de vista técnico, así que decidimos descartarlo. Además, el tiempo computacional que requeriría es
#otra limitación que ha estado siempre presente a lo largo del proyecto.

#SEGUNDO PLANTEAMIENTO. Experimentación y lluvia de ideas
#En esta fase estuve dandole vueltas a una manera de simplificar el problema o de resolverlo con más facilidades
#Pensé que podría ser buena idea entrenar un modelo que procesase únicamente texto (mucho más ligero que el audio), con idea de que hiciera lo siguiente:
# - Tomar la letra y los acordes de una canción (por separado)
# - Procesar la letra y predecir NO los acordes, sino solamente su posición en la letra.
#      Por qué conformarme con un planteamiento tan sencillo? Es que acaso no se supone que el objetivo es sacar los acordes?
#      En realidad el objetivo es mostrárselos al usuario en tiempo real. Lo que ocurre es que no encontramos ninguna web, app, etc que ofreciera esto
#      Hay que destacar que, como músicos aficionados, ninguno conocíamos una web/app que ofreciera las dos funcionalidades a la vez.
# - Una vez que el modelo haya predicho la posición de los acordes, mostrarlos al usuario en tiempo real

#Buscamos el visto bueno de este nuevo proyecto en el que tratamos de encontrar correlaciones entre la letra y la posición de los acordes, jugando
#también con los saltos de línea, palabras más frecuentes, etc.

#Durante este segundo planteamiento estuve trabajando en algoritmos con los que analizar la frecuencia de palabras y terminaciones más frecuentes o que 
#solían correlacionar con llevar un acorde encima, así como con algoritmos que fueran capaces de detectar estructuras en la letra de las canciones.

#En este punto trabajé con algoritmos relacionados tanto con teoría de la información como algoritmos generadores de esquemas que reflejen la estructura
#subyacente de las canciones, así como medidores de complejidad y entropía de las letras (entropía de Shannon + procesamiento de señales para encontrar
#periodicidades (Fourier))

import random
import matplotlib.pyplot as plt

def simplify_series(series):
    """
    Simplifico una serie a base de contar patrones iguales consecutivos (en este caso 0s y 1s).

    Args:
        series (list): Una lista que representa una letra de canción.

    Returns:
        int: Cantidad de pasos hasta simplificar la serie a un solo elemento.
    """
    steps = 0
    while len(series) > 1:
        new_series = []
        count = 1

        for i in range(1, len(series)):
            if series[i] == series[i - 1]:
                count += 1
            else:
                new_series.append(count)
                count = 1

        new_series.append(count)
        series = new_series
        steps += 1

    return steps

def generate_random_binary_series(length, count):
    """
    Aquí genero una lista binaria, la idea es representar la aleatoriedad del texto (que tiene menos entropía que randint y similares).

    Args:
        length (int): Longitud de cada serie/frase binaria.
        count (int): Cuántas frases va a generar.

    Returns:
        list: Devuelve una lista aleatoria.
    """
    return [[random.choice([0, 1]) for _ in range(length)] for _ in range(count)]

def main():
    #Me pide longitud de la serie y cuántas series generar
    length = int(input("Enter the length of the binary series: "))
    count = int(input("Enter the number of binary series to generate: "))

    # Y las genera
    binary_series_list = generate_random_binary_series(length, count)

    #Aquí simplifico las series y las guardo en una lista
    steps_list = []
    for series in binary_series_list:
        steps = simplify_series(series)
        steps_list.append(steps)

    #Le pido que me saque las tres últimas para comprobar que no haya errores
    print("Last three series analyzed:")
    for series, steps in zip(binary_series_list[-3:], steps_list[-3:]):
        print(f"Series: {series}, Steps: {steps}")

    #Saco los pasos promedio
    average_steps = sum(steps_list) / len(steps_list)
    print(f"Average steps to simplify: {average_steps:.2f}")

    #Y lo ploteo
    plt.hist(steps_list, bins=range(1, max(steps_list) + 2), edgecolor='black', align='left')
    plt.title("Distribution of Steps")
    plt.xlabel("Steps")
    plt.ylabel("Frequency")
    plt.xticks(range(1, max(steps_list) + 1))
    plt.show()

if __name__ == "__main__":
    main()
#La idea es analizar la entropía de las letras de las canciones, con idea de encontrar patrones que nos permitan predecir la posición de los acordes
#en la letra. Es importante conocer la entropía porque puede ser un indicador de la complejidad y permitirnos saber hasta qué punto las vamos a poder
#simplificar. Digo lo de simplificar porque otra cosa que preocupa es el tamaño del ds final.

#__________________________________________________________________________________________________________________________________________________________

#Además de esto, estuve trabajando también en un algoritmo que fuera capaz de comprimir el tamaño de los archivos mediante códigos de Huffman
#Este último quedó en fase experimental, pero dejo por aquí algunas pruebas:
#La idea es encontrar las palabras más frecuentes en las letras de las canciones para comprimirlas empleando su diccionario de frecuencias para
#optimizar el tamaño de cada código. Este es el compresor. Para descomprimirlo basta con el diccionario de códigos que genera el propio compresor

import heapq
from collections import defaultdict, namedtuple
import graphviz

class Node(namedtuple("Node", ["left", "right"])):
    def __repr__(self):
        return "Leaf" if not isinstance(self, Node) else "Node"

def huffman_tree(frequencies):
    heap = [ (freq, symbol) for symbol, freq in frequencies.items() ]
    heapq.heapify(heap)

    while len(heap) > 1:
        freq1, left = heapq.heappop(heap)
        freq2, right = heapq.heappop(heap)
        heapq.heappush(heap, (freq1 + freq2, Node(left, right)))

    return heap[0][1]

def generate_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}

    if isinstance(node, Node):
        generate_codes(node.left, prefix + "0", codebook)
        generate_codes(node.right, prefix + "1", codebook)
    else:
        codebook[node] = prefix

    return codebook

def average_length(codes, frequencies):
    total_length = sum(len(codes[symbol]) * freq for symbol, freq in frequencies.items())
    total_symbols = sum(frequencies.values())
    return total_length / total_symbols

def visualize_tree(node):
    def add_edges(graph, node, parent_name="", edge_label=""):
        if isinstance(node, Node):
            left_name = f"{id(node.left)}"
            right_name = f"{id(node.right)}"

            graph.node(left_name, label=str(node.left if isinstance(node.left, str) else ""))
            graph.node(right_name, label=str(node.right if isinstance(node.right, str) else ""))
            graph.edge(parent_name, left_name, label="0")
            graph.edge(parent_name, right_name, label="1")

            add_edges(graph, node.left, left_name)
            add_edges(graph, node.right, right_name)
        else:
            node_name = f"{id(node)}"
            graph.node(node_name, label=node)
            graph.edge(parent_name, node_name, label=edge_label)

    graph = graphviz.Digraph()
    root_name = f"{id(node)}"
    graph.node(root_name, label="Root")
    add_edges(graph, node, root_name)
    return graph

# Tabla completa de frecuencias
frequencies = {
    'de': 9999518, 'la': 6277560, 'que': 4681839, 'el': 4569652, 'en': 4234281,
    'y': 4180279, 'a': 3260939, 'los': 2618657, 'se': 2022514, 'del': 1857225,
    'las': 1686741, 'un': 1659827, 'por': 1561904, 'con': 1481607, 'no': 1465503,
    'una': 1347603, 'su': 1103617, 'para': 1062152, 'es': 1019669, 'al': 951054,
    'lo': 929333, 'como': 882725, 'más': 797765, 'pero': 770511, 'sus': 747644,
    'le': 725751, 'ya': 656055, 'o': 646856, 'este': 607282, 'sí': 599006,
    'porque': 597580, 'esta': 587503, 'entre': 565719, 'cuando': 511246,
    'muy': 480477, 'sin': 462938, 'sobre': 449549, 'también': 428582,
    'me': 425617, 'hasta': 405891, 'hay': 399401, 'donde': 394181, 'quien': 391873,
    'desde': 379377, 'todo': 374224, 'nos': 359025, 'durante': 353344,
    'todos': 346382, 'uno': 339324, 'les': 328949, 'ni': 323570, 'contra': 315688,
    'otros': 307069, 'ese': 305597, 'eso': 303995, 'ante': 291658, 'ellos': 287030,
    'e': 286518, 'esto': 281200, 'mí': 280314, 'antes': 277304, 'algunos': 275795,
    'qué': 272958, 'unos': 267012, 'yo': 266345, 'otro': 262001, 'otras': 259009,
    'otra': 256878, 'él': 251550, 'tanto': 249238, 'esa': 248991, 'estos': 246242,
    'mucho': 244742, 'quienes': 244581, 'nada': 241448, 'muchos': 240775,
    'cual': 236509, 'poco': 236496, 'ella': 235193, 'estar': 234926, 'estas': 234578,
    'algunas': 230501, 'algo': 230062, 'nosotros': 226287, 'mi': 225868,
    'mis': 224674, 'tus': 224281, 'ahora': 221748, 'cada': 220382, 'tú': 218499,
    'vosotros': 216331, 'ellas': 213231, 'usted': 211345, 'vuestra': 210230,
    'vuestras': 209172, 'vuestro': 207804, 'vuestros': 206450, 'ellas': 204382,
    'usted': 202231, 'ciertas': 200321, 'vosotros': 198221, 'ciertos': 196503, ' ':25000000
}

# Crear árbol de Huffman y generar códigos
huffman_root = huffman_tree(frequencies)
codes = generate_codes(huffman_root)

print("\nCódigos de Huffman:")
for symbol, code in codes.items():
    print(f"{symbol}: {code}")

# Calcular longitud promedio
avg_length = average_length(codes, frequencies)
print(f"\nLongitud promedio de un carácter: {avg_length:.2f}")

# Visualización del árbol
graph = visualize_tree(huffman_root)
graph.render("huffman_tree", format="png", cleanup=True)
print("\nEl árbol de Huffman se ha guardado como 'huffman_tree.png'.")

#__________________________________________________________________________________________________________________________________________________________


import heapq
from collections import defaultdict, namedtuple
import graphviz

class Node(namedtuple("Node", ["left", "right"])):
    def __repr__(self):
        return f"Node({self.left}, {self.right})"

def huffman_tree(frequencies):
    """
    Construye un árbol de Huffman a partir de un diccionario de frecuencias.

    :param frequencies: Diccionario donde las claves son elementos y los valores sus frecuencias
    :return: El nodo raíz del árbol de Huffman
    """
    heap = [ (freq, symbol) for symbol, freq in frequencies.items() ]
    heapq.heapify(heap)

    while len(heap) > 1:
        freq1, left = heapq.heappop(heap)
        freq2, right = heapq.heappop(heap)
        heapq.heappush(heap, (freq1 + freq2, Node(left, right)))

    return heap[0][1]  # El nodo raíz del árbol

def generate_codes(node, prefix="", codebook=None):
    """
    Genera los códigos binarios para cada símbolo en el árbol de Huffman.

    :param node: Nodo raíz del árbol de Huffman
    :param prefix: Prefijo acumulado del código binario
    :param codebook: Diccionario para almacenar los códigos generados
    :return: Diccionario con los códigos binarios
    """
    if codebook is None:
        codebook = {}

    if isinstance(node, Node):
        generate_codes(node.left, prefix + "0", codebook)
        generate_codes(node.right, prefix + "1", codebook)
    else:
        codebook[node] = prefix

    return codebook

def visualize_tree(node):
    """
    Visualiza el árbol de Huffman como un grafo utilizando Graphviz.

    :param node: Nodo raíz del árbol de Huffman
    :return: Objeto Digraph para renderizar
    """
    def add_edges(graph, node, parent_name="", edge_label=""):
        if isinstance(node, Node):
            left_name = f"{id(node.left)}"
            right_name = f"{id(node.right)}"

            graph.node(left_name, label=str(node.left))
            graph.node(right_name, label=str(node.right))
            graph.edge(parent_name, left_name, label="0")
            graph.edge(parent_name, right_name, label="1")

            add_edges(graph, node.left, left_name)
            add_edges(graph, node.right, right_name)
        else:
            node_name = f"{id(node)}"
            graph.node(node_name, label=str(node))
            graph.edge(parent_name, node_name, label=edge_label)

    graph = graphviz.Digraph()
    root_name = f"{id(node)}"
    graph.node(root_name, label=str(node))
    add_edges(graph, node, root_name)
    return graph

# Ejemplo de uso
frequencies = {
    '1': 5040,
    '2': 2520,
    '3': 1680,
    '4': 1260,
    '5': 1008,
    '6': 840,
    '7': 720,
    '8': 630,
    '9': 560,
    '10': 504,
    '11': 458,
    '12': 420,
    '13': 388,
    '14': 360,
    '15': 336,
    '16': 315,
    '17': 296,
    '18': 280,
    '19': 265,
    '20': 252,
    '21': 240,
    '22': 229,
    '23': 219,
    '24': 210,
    '25': 202,
    '26': 194,
    '27': 187,
    '28': 180
}

huffman_root = huffman_tree(frequencies)
codes = generate_codes(huffman_root)

print("Códigos de Huffman:")
for symbol, code in codes.items():
    print(f"{symbol}: {code}")

# Visualización del árbol
graph = visualize_tree(huffman_root)
from IPython.display import Image, display

graph.render("huffman_tree", format="png", cleanup=True)
display(Image("huffman_tree.png"))

#__________________________________________________________________________________________________________________________________________________________


#Más allá de estos algoritmos, dejo por aquí las fases en las que fue evolucionando el proyecto a sus incios, las cuales explican
#el porqué de que dejasemos de usar los algorimos que acabo de presentar



#FASE 1. DESCUBRIR EL DS Y EXPERIMENTAR CON ÉL

#Idea general: Crear un algoritmo que fuera capaz de hacer recomendaciones musicales buenas y relevantes para el usuario

#PRIMER PLANTEAMIENTO. Experimentación

#La primera idea consistió en idear un algoritmo que emplease tanto los datos de los gustos musicales de los usuarios como los datos de desempeño de
#las canciones en las listas de reproducción. 
#Los problemas que imposibilitaron la implementación de este algoritmo fueron:
# - El dataset que obteníamos de Spotify no contenía información sobre las listas de reproducción de los usuarios, por lo que no podíamos saber qué 
#   columnas de la matriz de gustos musicales correspondían con qué usuarios
# - Incluso aunque conociéramos los datos de otros usuarios, seguiríamos sin conocer los datos del usuario al que queremos recomendarle canciones


#SEGUNDO PLANTEAMIENTO

#La segunda idea consistió en emplear un algoritmo de recomendación basado en KNN. Es decir, la idea era ir descubriendo los gustos musicales del
#usuario con idea de ubicarlo dentro de un gran mapa de usuarios. Este mapa tiene tantas dimensiones como características estemos midiendo.
#De nuevo, volvimos a encontarnos con que no teníamos información sobre las listas de reproducción de los usuarios
#Por qué plantear un algoritmo de recomendación basado en KNN en el que se busque reubicar al usuario en un mapa de usuarios cuando ya sabemos que
#no disponemos de esos datos? Simplemente porque en el momento de plantearme este algoritmo aún no estaba claro qué datos íbamos a tener y cuáles no.

#TERCER PLANTEAMIENTO

#El tercer planteamiento es el definitivo! Este es el que se explica y detalla a lo largo de toda la memoria

In [ ]:

#Este código fue desarrollado con idea de testear un recomendador basado en el feedback del usuario. Empieza a hacer recomendaciones
#Neutras/impersonales o en general no necesariamente relevantes para el usuario y se va alimentando de su feedback
#En este caso se simulan las canciones con números aleatorios

import random

# Inicializar el diccionario para rastrear la frecuencia
frecuencias = {i: 0 for i in range(1, 101)}

def calcular_probabilidades(frecuencias):
    """Calcula las probabilidades basadas en las frecuencias."""
    # Sumar 1 a cada frecuencia
    frecuencias_ajustadas = {k: v + 1 for k, v in frecuencias.items()}
    # Calcular el total de las frecuencias ajustadas
    total = sum(frecuencias_ajustadas.values())
    # Calcular las probabilidades como frecuencia ajustada / total
    probabilidades = {k: v / total for k, v in frecuencias_ajustadas.items()}
    return probabilidades

def generar_numero_con_probabilidad(probabilidades):
    """Genera un número aleatorio basado en las probabilidades dadas."""
    numeros = list(probabilidades.keys())
    pesos = list(probabilidades.values())
    # Usar random.choices para generar un número con pesos
    return random.choices(numeros, weights=pesos, k=1)[0]

while True:
    # Calcular las probabilidades basadas en las frecuencias actuales
    probabilidades = calcular_probabilidades(frecuencias)

    # Generar tres números basados en las probabilidades calculadas
    numeros = [generar_numero_con_probabilidad(probabilidades) for _ in range(3)]
    print("\nNúmeros generados:", numeros)

    # Pedir al usuario que elija uno de los números o salir
    print("Escribe el número que quieres elegir o 'salir' para terminar:")
    eleccion = input("> ")

    if eleccion.lower() == 'salir':
        # Mostrar las frecuencias y probabilidades y salir
        print("\nFrecuencia de cada número:")
        for numero, frecuencia in frecuencias.items():
            print(f"{numero}: {frecuencia}")

        print("\nProbabilidades de cada número:")
        for numero, probabilidad in probabilidades.items():
            print(f"{numero}: {probabilidad:.4f}")
        break

    try:
        # Convertir la elección a un número entero
        eleccion = int(eleccion)
        if eleccion in numeros:
            # Incrementar la frecuencia del número elegido
            frecuencias[eleccion] += 1
            print(f"Has elegido el número {eleccion}.")
        else:
            print("Por favor, elige uno de los números mostrados.")
    except ValueError:
        print("Entrada inválida. Escribe un número o 'salir'.")
